In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import sys
sys.path.append("..")


from connector.cnxn import server_access
import pandas as pd

### Build Universe

- Identify stocks having Enterprise Value greater than 1 Billion. 
- Assuming that we need to find universe for 2017 we would look for companies based on financial reports of Q3'2016 EV.
- Q4'2016 data will be avaliable once Q1'2017 starts therefore we opt for Q3'2016.

In [2]:
engine = server_access()

universe_sql = '''
select symbol from tbl_historical_ev where date = '2016-09-30' and enterpriseValue > 1000000000 
order by enterpriseValue desc
'''

universe_df = pd.read_sql(universe_sql, engine)
universe_list = universe_df.symbol.to_list()

 At this point we have a total of 490 stocks in our universe, first step is to compute company's **earnings yield** $(EBIT ÷ Enterprise Value)$
 
 $EBIT = Net income + interest expenses + taxes$ \
 $EBITDA = Net income + interest expense + taxes + depreciation + amortization$

Based on above formula we derive EBIT from EBITDA using the below formula

$EBIT = EBITDA - depreciation -  amortization$

**Derive working capital** and **Net Fixed Assets**



$Working Capital = Current Assets - Current Liabilities$ \
$Net Fixed Assets = Total Fixed Assets – Accumulated Depreciation$ \
$Return On Capital = [EBIT ÷ (Net Fixed Assets + Working Capital)]$

In [3]:
data_sql = '''
select a.symbol,a.enterpriseValue, b.[totalCurrentAssets], b.[totalCurrentLiabilities], b.[propertyPlantEquipmentNet] fixedAsset, 
b.[accumulatedOtherComprehensiveIncomeLoss] accumulatedDepreciation, c.ebitda, c.[depreciationAndAmortization]
from tbl_historical_ev a
left join tbl_balance_sheet b on a.symbol = b.symbol
left join [dbo].[tbl_income_statement] c on a.symbol = c.symbol
where a.date = '2016-09-30' and a.enterpriseValue > 1000000000 and b.calendarYear = '2016' and b.period = 'Q3'
and c.calendarYear = '2016' and c.period = 'Q3'
order by  a.enterpriseValue desc
'''
df = pd.read_sql(data_sql, engine)

# convert to numeric
df.ebitda = df.ebitda.astype(float).fillna(0.0)
df.depreciationAndAmortization = df.depreciationAndAmortization.astype(float).fillna(0.0)
df.totalCurrentAssets = df.totalCurrentAssets.astype(float).fillna(0.0)
df.totalCurrentLiabilities = df.totalCurrentLiabilities.astype(float).fillna(0.0)
df.fixedAsset = df.fixedAsset.astype(float).fillna(0.0)
df.accumulatedDepreciation = df.accumulatedDepreciation.astype(float).fillna(0.0)

# compute earnings yield
df['earnings_yield'] = (df.ebitda - df.depreciationAndAmortization ) \
/ df['enterpriseValue'] 
df['ReturnOnCapital'] = (df.ebitda - df.depreciationAndAmortization ) \
/ ((df.fixedAsset - df.accumulatedDepreciation)  + (df.totalCurrentAssets - df.totalCurrentLiabilities))

In [4]:
# map all symbols with their respective industry
ticker_mapping_sql = '''
select * from tbl_tickers
'''
ticker_mapping = pd.read_sql(ticker_mapping_sql, engine)

analysis_df = df[['symbol', 'earnings_yield', 'ReturnOnCapital']]

analysis_df = analysis_df.set_index('symbol').join(ticker_mapping.set_index('Ticker')[['Industry', 'Sector', 
                                                                                       'Industry_Group', 'Classification']])
# remove utilites and financials companies
analysis_df = analysis_df[~analysis_df.Industry.isin(['UTILITIES','DIVERSIFIED FINANCIALS','BANKS'])]

# discard those for which industry is unknown
analysis_df = analysis_df[~analysis_df.Industry.isna()]


In [5]:
# sort values based on Earnings Yield and Return on Capital
analysis_df = analysis_df.sort_values(by=['earnings_yield', 'ReturnOnCapital'], ascending = False)

In [6]:
# pick top 30 tickers 
picked_symbols = analysis_df.index[:30]

In [7]:
analysis_df.head(30)

,earnings_yield,ReturnOnCapital,Industry,Sector,Industry_Group,Classification
GRVY,0.329351,0.111498,MEDIA & ENTERTAINMENT,COMMUNICATION SERVICES,Entertainment,D
VEON,0.312699,0.065102,TELECOMMUNICATION SERVICES,COMMUNICATION SERVICES,Wireless Telecommunication Services,D
CHRS,0.077994,0.663432,"PHARMACEUTICALS, BIOTECHNOLOGY & LIFE SCIENCES",HEALTH CARE,Biotechnology,D
UAL,0.064019,0.090308,TRANSPORTATION,INDUSTRIALS,Airlines,C
HA,0.062413,0.099858,TRANSPORTATION,INDUSTRIALS,Airlines,C
PENN,0.058316,0.057177,CONSUMER SERVICES,CONSUMER DISCRETIONARY,"Hotels, Restaurants & Leisure",C
UTHR,0.057762,0.151973,"PHARMACEUTICALS, BIOTECHNOLOGY & LIFE SCIENCES",HEALTH CARE,Biotechnology,D
ESGR,0.057107,9.804015,INSURANCE,FINANCIALS,Insurance,C
IDCC,0.056556,0.211524,SOFTWARE & SERVICES,INFORMATION TECHNOLOGY,Software,C
IAC,0.056430,0.057331,MEDIA & ENTERTAINMENT,COMMUNICATION SERVICES,Interactive Media & Services,D
